In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn import metrics

In [5]:
df =pd.read_csv('mergedfinal.csv')

In [6]:
z=df['DepDel15']
y=df['ArrDel15']
X=df.drop(['Unnamed: 0','Unnamed: 0_x','DepDel15','ArrDel15','Unnamed: 0_y','FlightDate','Origin','Dest','date','place_x','place_y','DepTime','ArrDelayMinutes','ArrTime','time_x','time_y','Arrtime1','Deptime1','Unnamed: 0.1','DepDelayMinutes'],axis=1)
# X=df.drop(['Unnamed: 0','Unnamed: 0_x','ArrDel15','Unnamed: 0_y','FlightDate','Origin','Dest_x','date','Dest_y','ArrDelayMinutes','ArrTime'],axis=1)


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851334 entries, 0 to 1851333
Data columns (total 32 columns):
Quarter            int64
Year               int64
Month              int64
DayofMonth         int64
CRSDepTime         int64
OriginAirportID    int64
DestAirportID      int64
CRSArrTime         int64
windspeedKmph_x    int64
weatherCode_x      int64
precipMM_x         float64
WindGustKmph_x     int64
pressure_x         int64
cloudcover_x       int64
winddirDegree_x    int64
humidity_x         int64
cloudcover.1_x     int64
DewPointF_x        int64
tempF_x            int64
WindChillF_x       int64
windspeedKmph_y    int64
weatherCode_y      int64
precipMM_y         float64
WindGustKmph_y     int64
pressure_y         int64
cloudcover_y       int64
winddirDegree_y    int64
humidity_y         int64
cloudcover.1_y     int64
DewPointF_y        int64
tempF_y            int64
WindChillF_y       int64
dtypes: float64(2), int64(30)
memory usage: 452.0 MB


In [23]:
df.loc[df['ArrDel15'].isin([0])].count()


Unnamed: 0         1463295
Unnamed: 0_x       1463295
FlightDate         1463295
Quarter            1463295
Year               1463295
Month              1463295
DayofMonth         1463295
DepTime            1463295
DepDel15           1463295
CRSDepTime         1463295
DepDelayMinutes    1463295
OriginAirportID    1463295
DestAirportID      1463295
ArrTime            1463295
CRSArrTime         1463295
ArrDel15           1463295
ArrDelayMinutes    1463295
Origin             1463295
Dest               1463295
date               1463295
Arrtime1           1463295
Deptime1           1463295
Unnamed: 0_y       1463295
windspeedKmph_x    1463295
weatherCode_x      1463295
precipMM_x         1463295
WindGustKmph_x     1463295
pressure_x         1463295
cloudcover_x       1463295
winddirDegree_x    1463295
humidity_x         1463295
cloudcover.1_x     1463295
DewPointF_x        1463295
tempF_x            1463295
time_x             1463295
WindChillF_x       1463295
place_x            1463295
U

In [18]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0.0, 1170542), (1.0, 1170542)]


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [40]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[(0.0, 310647), (1.0, 310647)]


In [52]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_res,y_res = sm.fit_sample(X_train,y_train)

In [47]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_jobs=-1)
etc.fit(X_resampled, y_resampled)
predictions = etc.predict(X_test)
print(metrics.accuracy_score(y_test,predictions))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(etc.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(etc.score(X_test, y_test)))

c:\users\kevin thelly\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.681022073260647
Accuracy of Decision Tree classifier on training set: 0.83
Accuracy of Decision Tree classifier on test set: 0.68


In [48]:
import collections
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

[[207270  85605]
 [ 32502  44890]]


In [49]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = predictions
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.86      0.71      0.78    292875
     class 1       0.34      0.58      0.43     77392

   micro avg       0.68      0.68      0.68    370267
   macro avg       0.60      0.64      0.61    370267
weighted avg       0.76      0.68      0.71    370267



In [21]:
#FEATURE IMPORTANCE
import numpy as np
importances=etc.feature_importances_
std= np.std([tree.feature_importances_ for tree in etc.estimators_],axis=0)
indices=np.argsort(importances)[::-1]

In [22]:
#PRINTING FEATURE IMPORTANCE
print("Feature Ranking")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1,indices[f],importances[indices[f]]))

Feature Ranking
1. feature 5 (0.398379)
2. feature 8 (0.043005)
3. feature 4 (0.042176)
4. feature 12 (0.020588)
5. feature 24 (0.020135)
6. feature 27 (0.020094)
7. feature 16 (0.020083)
8. feature 31 (0.020047)
9. feature 15 (0.020037)
10. feature 21 (0.019964)
11. feature 28 (0.019732)
12. feature 32 (0.019664)
13. feature 9 (0.019656)
14. feature 18 (0.019227)
15. feature 25 (0.019153)
16. feature 30 (0.019084)
17. feature 13 (0.019020)
18. feature 19 (0.018935)
19. feature 20 (0.018906)
20. feature 7 (0.017876)
21. feature 22 (0.017486)
22. feature 3 (0.017140)
23. feature 6 (0.016966)
24. feature 14 (0.015738)
25. feature 26 (0.015613)
26. feature 10 (0.015537)
27. feature 29 (0.015365)
28. feature 17 (0.015240)
29. feature 2 (0.014063)
30. feature 11 (0.012633)
31. feature 23 (0.012171)
32. feature 0 (0.008472)
33. feature 1 (0.007815)


In [24]:
X.columns[1]

'Year'

In [10]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression( class_weight='balanced')
logisticRegr.fit(X_train, y_train)

predictionsLR = logisticRegr.predict(X_test)


c:\users\kevin thelly\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = predictionsLR
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

In [43]:
#Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_resampled, y_resampled)
predictionsDTC = clf.predict(X_test)

print(metrics.accuracy_score(y_test,predictions))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

0.6822158064315751
Accuracy of Decision Tree classifier on training set: 0.76
Accuracy of Decision Tree classifier on test set: 0.60


In [44]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = predictionsDTC
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.59      0.70    292875
     class 1       0.28      0.61      0.39     77392

   micro avg       0.60      0.60      0.60    370267
   macro avg       0.57      0.60      0.54    370267
weighted avg       0.73      0.60      0.63    370267



In [55]:
#Gaussian

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictionsgnb = gnb.predict(X_test)
print(metrics.accuracy_score(y_test,predictionsgnb))
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

0.8875217073085098
Accuracy of GNB classifier on training set: 0.89
Accuracy of GNB classifier on test set: 0.89


In [56]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = predictionsgnb
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.93      0.93      0.93    292660
     class 1       0.73      0.74      0.73     77607

   micro avg       0.89      0.89      0.89    370267
   macro avg       0.83      0.83      0.83    370267
weighted avg       0.89      0.89      0.89    370267



In [45]:
X_test_np=X_test.values

In [46]:
#XGBOOST
from xgboost import XGBClassifier
gbc=XGBClassifier()
gbc.fit(X_resampled,y_resampled)
predictions=gbc.predict(X_test)
print(metrics.accuracy_score(y_test,predictions))

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31'] ['Quarter', 'Year', 'Month', 'DayofMonth', 'CRSDepTime', 'OriginAirportID', 'DestAirportID', 'CRSArrTime', 'windspeedKmph_x', 'weatherCode_x', 'precipMM_x', 'WindGustKmph_x', 'pressure_x', 'cloudcover_x', 'winddirDegree_x', 'humidity_x', 'cloudcover.1_x', 'DewPointF_x', 'tempF_x', 'WindChillF_x', 'windspeedKmph_y', 'weatherCode_y', 'precipMM_y', 'WindGustKmph_y', 'pressure_y', 'cloudcover_y', 'winddirDegree_y', 'humidity_y', 'cloudcover.1_y', 'DewPointF_y', 'tempF_y', 'WindChillF_y']
expected f12, f28, f26, f21, f0, f5, f8, f9, f11, f29, f24, f20, f2, f25, f30, f16, f27, f6, f15, f19, f22, f1, f31, f14, f13, f3, f17, f4, f10, f18, f23, f7 in input data
training data did not have the following fields: pressure_x, Month, cloudcover_y, precip

In [34]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = predictions
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.80      0.99      0.88    292539
     class 1       0.68      0.05      0.09     77728

   micro avg       0.80      0.80      0.80    370267
   macro avg       0.74      0.52      0.49    370267
weighted avg       0.77      0.80      0.72    370267



In [34]:
predictionsgnb = gnb.predict(X)
X1=X
X1['NewArr15']=predictionsgnb
X1

,Quarter,Year,Month,DayofMonth,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,...,WindGustKmph_y,pressure_y,cloudcover_y,winddirDegree_y,humidity_y,cloudcover.1_y,DewPointF_y,tempF_y,WindChillF_y,NewArr15
0,1,2016,1,1,745,0.0,14747,12478,1602,22,...,8,1030,0,107,66,0,23,34,29,0.0
1,1,2016,1,1,705,0.0,14747,12892,940,12,...,8,1030,0,107,66,0,23,34,29,0.0
2,1,2016,1,1,717,6.0,14747,12478,1530,21,...,8,1030,0,107,66,0,23,34,29,0.0
3,1,2016,1,1,705,0.0,14747,14107,1057,10,...,8,1030,0,107,66,0,23,34,29,0.0
4,1,2016,1,1,759,0.0,14747,11292,1143,9,...,8,1030,0,107,66,0,23,34,29,0.0
5,1,2016,1,1,730,0.0,14747,12892,1019,10,...,8,1030,0,107,66,0,23,34,29,0.0
6,1,2016,1,1,724,35.0,14747,14771,946,19,...,8,1030,0,107,66,0,23,34,29,1.0
7,1,2016,1,1,738,2.0,14747,12266,1351,19,...,8,1030,0,107,66,0,23,34,29,0.0
8,1,2016,1,1,745,0.0,14747,10397,1527,17,...,8,1030,0,107,66,0,23,34,29,0.0
9,1,2016,1,1,705,40.0,14747,12889,930,8,...,8,1030,0,107,66,0,23,34,29,1.0


In [45]:
z=df['DepDel15']
y=df['ArrDel15']
X1=df.drop(['Unnamed: 0','Unnamed: 0_x','DepDel15','ArrDel15','Unnamed: 0_y','FlightDate','Origin','Dest','date','place_x','place_y','DepTime','ArrTime','time_x','time_y','Arrtime1','Deptime1','Unnamed: 0.1'],axis=1)
# X=df.drop(['Unnamed: 0','Unnamed: 0_x','ArrDel15','Unnamed: 0_y','FlightDate','Origin','Dest_x','date','Dest_y','ArrDelayMinutes','ArrTime'],axis=1)


In [51]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851334 entries, 0 to 1851333
Data columns (total 35 columns):
Quarter            int64
Year               int64
Month              int64
DayofMonth         int64
CRSDepTime         int64
DepDelayMinutes    float64
OriginAirportID    int64
DestAirportID      int64
CRSArrTime         int64
ArrDelayMinutes    float64
windspeedKmph_x    int64
weatherCode_x      int64
precipMM_x         float64
WindGustKmph_x     int64
pressure_x         int64
cloudcover_x       int64
winddirDegree_x    int64
humidity_x         int64
cloudcover.1_x     int64
DewPointF_x        int64
tempF_x            int64
WindChillF_x       int64
windspeedKmph_y    int64
weatherCode_y      int64
precipMM_y         float64
WindGustKmph_y     int64
pressure_y         int64
cloudcover_y       int64
winddirDegree_y    int64
humidity_y         int64
cloudcover.1_y     int64
DewPointF_y        int64
tempF_y            int64
WindChillF_y       int64
NewArr15           float64
dt

In [68]:
X_np=X.values

In [69]:
predictionsg= gbc.predict(X_np)
X1=df.drop(['Unnamed: 0','Unnamed: 0_x','DepDel15','ArrDel15','Unnamed: 0_y','FlightDate','Origin','Dest','date','place_x','place_y','DepTime','ArrTime','time_x','time_y','Arrtime1','Deptime1','Unnamed: 0.1'],axis=1)
X1['NewArr15']=predictionsg
result1=X1.loc[X1['NewArr15']==1]
result1.to_csv('module2resultextree.csv')